# Gabe Autonomous Slack Agent — Setup & Validation Notebook


Use this guided notebook to provision Slack resources, collect tokens/IDs, configure the local environment, and self-test the Gabe agent from within VS Code.

## 1. Choose the Notebook Kernel


1. In VS Code, press <kbd>⌘⇧P</kbd> (or <kbd>Ctrl⇧P</kbd> on Windows/Linux) and run **Python: Select Interpreter**.

2. Pick the `.venv` you created for this repo (path should end with `gabe-autonomous-slack-agent/.venv`).

3. After selecting, confirm the kernel indicator in the top-right of the notebook shows that environment.



> If the `.venv` doesn't exist yet, create it in a terminal first:

> ```bash

> python3 -m venv .venv

> .venv/bin/pip install -r requirements.txt

> ```

## 2. Prep Slack Accounts & Workspaces


- **Dedicated bot account**: create a separate Slack user (e.g., `gabe-bot`) that will own the Slack App and join TrustedWork channels.

- **TrustedWork workspaces**: ensure you're a Workspace Owner/Admin (or have one available) in each workspace where Gabe should operate.

- **Decide Gabe's home channel**: choose the channel where the agent should post heartbeats/status updates.



Keep the browser tabs for each workspace handy—you'll reference them in the provisioning steps below.

## 3. Provision the Slack App (one-time)


1. Browse to <https://api.slack.com/apps> and click **Create New App → From scratch**.

2. Name it (e.g., `Gabe Autonomous Agent`) and select the dedicated bot workspace.

3. Inside **Basic Information**:

   - Scroll to **App-Level Tokens** → Create token with scope `connections:write` (this is `SLACK_APP_TOKEN`).

   - In **App Credentials**, click **Show** to copy the **Signing Secret** (`SLACK_SIGNING_SECRET`).

4. Enable **Socket Mode** in the left nav.

5. Under **OAuth & Permissions** → **Scopes (Bot Token Scopes)** add:

   - `app_mentions:read`, `channels:history`, `chat:write`, `commands`,

   - `groups:history`, `im:history`, `im:read`, `im:write`, `mpim:history`.

6. Click **Install App to Workspace**. Copy the **Bot User OAuth Token** (`SLACK_BOT_TOKEN`).

7. Invite the bot user into the home channel in Slack (`/invite @gabe`).



Repeat the install for each TrustedWork workspace where the agent should live.

## 4. Collect IDs & Tokens


| Item | Where to find it | Notes |

| --- | --- | --- |

| `SLACK_SIGNING_SECRET` | App → **Basic Information → App Credentials** | Click **Show** then copy. |

| `SLACK_APP_TOKEN` | App → **Basic Information → App-Level Tokens** | Create if needed, scope must include `connections:write`. |

| `SLACK_BOT_TOKEN` | App → **OAuth & Permissions → Bot User OAuth Token** | Starts with `xoxb-`. Reinstall if rotated. |

| Channel ID | In Slack, open channel → `View channel details` → scroll down (ID starts with `C`). OR copy channel link and grab the trailing `C...` portion. |

| Workspace / team ID | Any Slack URL starts with `https://app.slack.com/client/T.../` → that `T...` value is the team ID. You can also run `curl -H "Authorization: Bearer $SLACK_BOT_TOKEN" https://slack.com/api/auth.test`. |



Once gathered, populate the project `.env` (copy `config/.env.example` if you haven't done so).

In [ ]:
# Optional: ensure required libraries are available in the selected kernel.
# Skip this cell if you've already pip-installed the project requirements.
try:
    import slack_sdk  # noqa: F401
    import dotenv  # noqa: F401
except ImportError:
    import subprocess
    import sys

    subprocess.check_call([
        sys.executable,
        "-m",
        "pip",
        "install",
        "slack-sdk",
        "python-dotenv",
    ])

In [ ]:
from pathlib import Path
from dotenv import load_dotenv

PROJECT_ROOT = Path.cwd().parent  # notebook lives in notebooks/
ENV_FILE = PROJECT_ROOT / ".env"

if ENV_FILE.exists():
    load_dotenv(ENV_FILE)
    print(f"Loaded environment variables from {ENV_FILE}")
else:
    print(
        "⚠️ No .env file found at project root. Run `cp config/.env.example .env` and fill in your Slack secrets."
    )

In [ ]:
import os
from textwrap import indent

REQUIRED_VARS = [
    "SLACK_BOT_TOKEN",
    "SLACK_APP_TOKEN",
    "SLACK_SIGNING_SECRET",
]
OPTIONAL_VARS = [
    "GABE_HOME_CHANNEL",
    "TRUSTED_WORKSPACE_IDS",
]

missing = [name for name in REQUIRED_VARS if not os.environ.get(name)]
if missing:
    raise RuntimeError(
        "Missing required environment variables:\n" + indent("\n".join(missing), "  • ")
    )

print("✅ Required Slack environment variables are present.")
for name in OPTIONAL_VARS:
    value = os.environ.get(name)
    if value:
        print(f"• {name} = {value}")
    else:
        print(f"• {name} not set (optional)")

In [ ]:
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

bot_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=bot_token)

try:
    auth_info = client.auth_test()
except SlackApiError as exc:  # pragma: no cover - interactive cell
    print("❌ Slack auth.test failed:")
    print(json.dumps(exc.response.data, indent=2))
else:
    print("✅ auth.test succeeded")
    print(json.dumps(auth_info.data, indent=2))

In [ ]:
from itertools import islice

home_channel = os.environ.get("GABE_HOME_CHANNEL")
team_filter = set(filter(None, (os.environ.get("TRUSTED_WORKSPACE_IDS") or "").split(",")))

print("Known workspaces (from env):", team_filter or "<none declared>")

try:
    conversations = client.conversations_list(types="public_channel,private_channel")
except SlackApiError as exc:  # pragma: no cover - interactive cell
    print("⚠️ conversations.list failed (ensure bot has channels:history & groups:history scopes)")
    print(json.dumps(exc.response.data, indent=2))
else:
    channels = conversations.get("channels", [])
    if not channels:
        print("No channels returned; invite the bot to at least one channel.")
    else:
        def describe(channel):
            marker = "<- home channel" if home_channel and channel["id"] == home_channel else ""
            return f"{channel['name']} ({channel['id']}) {marker}"

        print("First few channels I can see:")
        for entry in islice(channels, 10):
            print(" •", describe(entry))

## 5. Run Local Self-Tests


Once the environment variables are set and Slack checks succeed, run the project's unit tests to ensure core logic is behaving.

In [ ]:
import subprocess
import sys

result = subprocess.run([
    sys.executable,
    "-m",
    "pytest",
    "tests",
], capture_output=True, text=True)

print(result.stdout)
if result.returncode != 0:
    print(result.stderr)
    raise SystemExit(result.returncode)

## 6. Next Steps


- Launch Gabe locally with `python -m src.gabe_bot`.

- Deploy the agent to your preferred runtime (e.g., Fly.io, AWS Lambda, or a container runner) and copy the same `.env` variables.

- As you add new capabilities, extend this notebook with additional validation checks.



Feel free to rerun the Slack verification cells any time you rotate tokens or move Gabe to a new workspace.